## Basis Set Convergence

plotting Dimerization-energy vs cardinal number for HF, MP2 and TPSS_D3


In [ ]:
# load data: Monomer

with open('/Dimerization_energies', 'r') as f:
    lines = f.readlines()


# load data: Dimer

with open('/Dimerization_energies', 'r') as f:
    lines = f.readlines()